# Conway's Game of Life

The [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) is a cellular automaton devised by the British mathematician John Horton Conway in 1970. It is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One only interacts with the Game of Life by creating an initial configuration.

The universe of the Game of Life is an infinite, two-dimensional orthogonal grid of square cells, each of which is in one of two possible states: live or dead. Every cell interacts with its eight neighbours. The game evolves. At each time step, the following transitions occur:

1. Any live cell with exactly two or three live neighbours survives.
2. Any dead cell with exactly three live neighbours becomes a live cell.
3. All other live cells die in the next generation. All other dead cells stay dead.

The first generation must be initialized. Every new generation is created by applying the above rules simultaneously to every cell in the previous generations; births and deaths occur simultaneously. The moment when this happens is called a tick. Since every generation depends only on the previous one, this process is a [Markov chain](https://en.wikipedia.org/wiki/Markov_chain).

The following few exercises will implement the Game of Life. We will consider a finite universe with periodic boundary conditions.

### Exercise:

Write a function `neighbours` that returns the number of live neighbours of a cell. The function should accept the `world` matrix of boolean values representing the state of all cells (`true` if the cell is alive and `false` otherwise) and the index of the row and column of the cell.

**Hint:** Use the following properties of the `mod1` function to implement periodic boundaries.

```julia
mod1(1, 4)
mod1(4, 4)
mod1(5, 4)
```

**Bonus:** Implement a more general function which computes the number of alive cells in a neighbourhood of a given size.

In [ ]:
mod1(1, 4)
mod1(4, 4)
mod1(5, 4)

**Example initialization**

In [ ]:
world = zeros(Bool, 15, 15)
row, col = 5, 5

world[row, col] = 1
world[row, col + 1] = 1
world[row - 1, col + 6] = 1
world[row + 1, col + 1] = 1
world[row + 1, col + 5] = 1
world[row + 1, col + 6] = 1
world[row + 1, col + 7] = 1

heatmap(world; axis = nothing, border = :none, cbar = false, ratio = :equal, size=(300,300))

#### Solution

One way to define the `neighbours` function is to check all neighbours manually.

In [ ]:
function neighbours(world, row, col)
    n, m = size(world)

    # this implements periodic boundaries
    down  = mod1(row + 1, n)
    up    = mod1(row - 1, n)
    left  = mod1(col - 1, m)
    right = mod1(col + 1, m)

    return (world[up, left]   + world[up, col]   + world[up, right]
          + world[row, left]                     + world[row, right]
          + world[down, left] + world[down, col] + world[down, right])
end

The approach above cannot define a general version of the `neighbours` function. In this case, we can use array indexing and loops. First, we compute proper row indices by `mod1` combined with the range operator.

In [ ]:
r = 1
rows = mod1.(row .+ (-r:r), size(world, 1))

Column indices can be computed similarly. Then we use nested loops to iterate through both rows and columns. Since the iteration includes the middle cell, we need to subtract its state.

In [ ]:
function neighbours(world, row, col; r = 1)
    rows = mod1.(row .+ (-r:r), size(world, 1))
    cols = mod1.(col .+ (-r:r), size(world, 2))

    return sum(world[i, j] for i in rows, j in cols) - world[row, col]
end


### Exercise:

Add a new method to the `neighbours` function that, for the `world` matrix, returns a matrix containing the numbers of living neighbours for each cell.

#### Solution

We created a function that computes the number of living neighbours for a single cell in the exercise above. One way to create a matrix with numbers of living neighbours is:

In [ ]:
function neighbours(world)
    n, m = size(world)
    return [neighbours(world, row, col) for row in 1:n, col in 1:m]
end

This is an example of multiple dispatch. The function `neighbours` can have both one and three input arguments.

### Exercise:

Write a function `willsurvive` that returns `true` if the cell will survive based on the conditions described at the beginning of the section and `false` otherwise. This function should accept two arguments: the state of the cell (`true`/`false`) and the number of living neighbours.

#### Solution

This function can be written using the `if-elseif-else` statement. Since `cell` is a boolean value, we do not need to compare with `true` or `false` explicitly.

In [ ]:
function willsurvive(cell, k)
    if k == 3
        return true
    elseif k == 2 && cell
        return true
    else
        return false
    end
end

We can write this function in a simpler form. We first realize that the short-circuit evaluation can merge the first two conditions. Since the function returns only `true` or `false`, we can write the function on one line.

In [ ]:
willsurvive(cell, k) = k == 3 || (k == 2 && cell)

### Exercise:

Combine these functions to write a function `evolve!` that evolves the given `world` matrix into a new generation.

#### Solution

We first compute the matrix with the numbers of living neighbours. Then we iterate over all elements of the `world` matrix and compute new states of all elements with the `willsurvive` function. Since we computed the number of living neighbours before iterating, we can overwrite the `world` matrix in place.


In [ ]:
function evolve!(world)
    ks = neighbours(world)
    for i in eachindex(world)
        world[i] = willsurvive(world[i], ks[i])
    end
    return
end

### Evaluation and visualization

In the four exercises above, we defined functions sufficient to animate the Game of Life. Use the following code to initialize the `world`.

In [ ]:
world = zeros(Bool, 30, 30)
row, col = 15, 15

world[row, col] = true
world[row, col + 1] = true
world[row - 1, col + 6] = true
world[row + 1, col + 1] = true
world[row + 1, col + 5] = true
world[row + 1, col + 6] = true
world[row + 1, col + 7] = true

We use the `Plots` package to create animations.

In [ ]:
using Plots

anim = @animate for i in 1:150
    heatmap(world; axis = nothing, border = :none, cbar = false, ratio = 1)
    evolve!(world)
end
gif(anim, "gameoflife.gif"; fps = 10)

After running the code above, the animation should display below (if the environment supports GIF display). If not, you can open the `gameoflife.gif` file generated in your working directory to view the animation.

Many different types of patterns occur in the Game of Life. For example, the following initialization is called the *pulsar*.

In [17]:
world = zeros(Bool, 17, 17)
line = zeros(17)
line[5:7] .= true
line[11:13] .= true

for ind in [3, 8, 10, 15]
    world[ind, :] .= line
    world[:, ind] .= line
end

We can animate the pulsar pattern using the same `evolve!` function.

In [ ]:
anim = @animate for i in 1:50
    heatmap(world; axis = nothing, border = :none, cbar = false, ratio = 1)
    evolve!(world)
end
gif(anim, "gameoflife_pulsar.gif"; fps = 10)

After running the code above, the pulsar animation should display below (if the environment supports GIF display). Otherwise, you can open the `gameoflife_pulsar.gif` file generated in your working directory to view the animation.